<a href="https://colab.research.google.com/github/vijay313v/airbnb.eda/blob/main/cardiac_chd_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#loading dataset
cv_df =pd.read_csv(r"/content/data_cardiovascular_risk.csv")

In [ ]:
# first 5 observations
cv_df.head()

In [ ]:
# shape of dataset
cv_df.shape

**3390 observation and 17 columns**

In [ ]:
#summary of datset
cv_df.info()

In [ ]:
#checking for null values
cv_df.isnull().sum()

In [ ]:
#statistical description of dataframe
cv_df.describe()

In [ ]:
#Checking the distribution of the target variable
cv_df['TenYearCHD'].value_counts()

# **EDA :**

## **Which sex is most likely to suffer from positive CHD.**

In [ ]:
# Analysis of sex column
plt.figure(figsize=(8, 6))

# Create a custom palette
my_palette = {0: 'blue', 1: 'orange'}

# Create the countplot
ax = sns.countplot(x=cv_df['sex'], hue=cv_df['TenYearCHD'], palette=my_palette)

plt.title('Gender more prone to CHD')
plt.legend(['No Risk', 'At Risk'])

# Calculate and add percentage text to the plot
total = len(cv_df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2, height + 5, f'{height/total*100:.2f}%', ha='center')

plt.show()


**Being a Male has high chances of CHD compare to Female.**

## **Smoking effect on CHD**

In [ ]:
#Analysis on cigs per day column
plt.figure(figsize=(12, 12))

# Create a custom palette
my_palette = {0: 'blue', 1: 'orange'}

# Create the countplot
ax = sns.countplot(x=cv_df['cigsPerDay'], hue=cv_df['TenYearCHD'], palette=my_palette)

plt.title('How much smoking cig per day has risk to CHD')
plt.legend(['No Risk', 'At Risk'])

plt.show()

## **Which Age people has high chances of positive CHD.**

In [ ]:
#Analysis on Age column
plt.figure(figsize=(8, 6))

# Create a custom palette
my_palette = {0: 'blue', 1: 'orange'}

# Create the countplot
ax = sns.countplot(x=cv_df['age'], hue=cv_df['TenYearCHD'], palette=my_palette)

plt.title(' Which Age more prone to CHD')
plt.legend(['No Risk', 'At Risk'])

plt.show()

**Age between 47 to 65 has high chances of positive CHD**

## **Does people taking BPMeds effect on CHD**

In [ ]:
# Analysis on BPMeds
plt.figure(figsize=(8, 6))

# Create a custom palette
my_palette = {0: 'blue', 1: 'orange'}

# Create the countplot
ax = sns.countplot(x=cv_df['BPMeds'], hue=cv_df['TenYearCHD'], palette=my_palette)

plt.title(' ')
plt.legend(['No Risk', 'At Risk'])

# Calculate and add percentage text to the plot
total = len(cv_df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2, height + 5, f'{height/total*100:.2f}%', ha='center')

plt.show()

**BPMeds increase the chances of CHD**

## **How does prevalent Stroke effect in positive CHD factor.**

In [ ]:
#Analysis on prevalent stroke column
plt.figure(figsize=(8, 6))

# Create a custom palette
my_palette = {0: 'blue', 1: 'orange'}

# Create the countplot
ax = sns.countplot(x=cv_df['prevalentStroke'], hue=cv_df['TenYearCHD'], palette=my_palette)

plt.title(' IS prevalent Stroke effect in CHD')
plt.legend(['No Risk', 'At Risk'])

# Calculate and add percentage text to the plot
total = len(cv_df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2, height + 5, f'{height/total*100:.2f}%', ha='center')
plt.show()

**Prevalent stroke increases chances of CHD in future.**

# **Does Prevalent Hypertension has effect on positive CHD in future**

In [ ]:
#Analysis on  prevalent Hpypertension column
plt.figure(figsize=(8, 6))

# Create a custom palette
my_palette = {0: 'blue', 1: 'orange'}

# Create the countplot
ax = sns.countplot(x=cv_df['prevalentHyp'], hue=cv_df['TenYearCHD'], palette=my_palette)

plt.title(' Effect of prevalent Hpypertension on CHD')
plt.legend(['No Risk', 'At Risk'])

# Calculate and add percentage text to the plot
total = len(cv_df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2, height + 5, f'{height/total*100:.2f}%', ha='center')

plt.show()

**prevalent Hypertension increases chances of CHD in patients**

# **Does Diabetes affect the chances of having a positive CHD risk factor**

In [ ]:
#Analysis of Diabetes column
plt.figure(figsize=(8, 6))

# Create a custom palette
my_palette = {0: 'blue', 1: 'orange'}

# Create the countplot
ax = sns.countplot(x=cv_df['diabetes'], hue=cv_df['TenYearCHD'], palette=my_palette)

plt.title(' Effect of Diabetes on CHD')
plt.legend(['No Risk', 'At Risk'])

# Calculate and add percentage text to the plot
total = len(cv_df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2, height + 5, f'{height/total*100:.2f}%', ha='center')

plt.show()

**Diabetes increases chances of CHD**

# **Data Cleaning**

In [ ]:
#Checking for null values
cv_df.isnull().sum()

In [ ]:
# List of columns with missing values that have to fill with mean
columns_to_fill = ['BPMeds',  'glucose']

# Iterate through the columns and fill NaN values with the mean of each column
for col in columns_to_fill:
    mean_value = cv_df[col].mean()
    cv_df[col].fillna(mean_value, inplace=True)

In [ ]:
cv_df.dropna(inplace=True)
cv_df.drop('id', axis=1, inplace=True)

In [ ]:
# Removing duplicate values
cv_df.drop_duplicates(inplace=True)


# **Outliers**

In [ ]:

# List with numerical columns that have outliers
outlier_cols = ['cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']

# Visualizing boxplots to find if columns contain outliers
plt.figure(figsize=(15, 10))  # Adjust the figsize as needed

for index, item in enumerate(outlier_cols):
    plt.subplot(2, 4, index + 1)
    sns.boxplot(x=cv_df[item])  # Access the column using cv_df[item]
    plt.title(item)

plt.tight_layout()
plt.show()

**From above boxplot we can infer that these columns have outliers, but in practicality even though they are not normal observations but it is still possible.So,ruling out such possible scenarios can be harmful for our future predictions. Therefore i will allow these outliers to be in dataset.**

# **Feature Engineering :**

**Feature Encoding :**

Machine Learning model work with numerical values therefore categorical columns have to converted/encoded into numerical variables.This process is known as Feature Encoding

Here we have two columns that require encoding and they are "sex" and "is_smoking".

In [ ]:
#Encoding the categorical columns
cv_df['sex'] = cv_df['sex'].apply(lambda x: 1 if x == 'M' else 0)
cv_df['is_smoking'] = cv_df['is_smoking'].apply(lambda x: 1 if x == 'YES' else 0)

# **Grouping columns for better Understanding :**

In [ ]:
def smoke_pattern (cigperday:float):
  """A function that returns the Smoking level
     by taking cigarettes per day as an input."""

  if cigperday==0:                    #Non smoker
    return 1
  elif cigperday>0 and cigperday<=10:       #Smoker with more than 0 and less than 10 cigs per day
    return 2
  elif cigperday>10 and cigperday<=20:      #Smoker with more than 10 and less than 20 cigs per day
    return 3
  elif cigperday>20 and cigperday<=30:      #Smoker with more than 20 and less than 30 cigs per day
    return 4
  elif cigperday>30 and cigperday<=40:      #Smoker with more than 30 and less than 40 cigs per day
    return 5
  else:                         #Smoker with more than 40 cigs per day
    return 6


In [ ]:

#Creating the Smokepattern column
cv_df['smoke_pattern'] = cv_df['cigsPerDay'].apply(lambda x: smoke_pattern(x))

In [ ]:

#Removing columns upon whom grouping has been done
cv_df.drop(columns={'is_smoking','cigsPerDay'},axis=1,inplace=True)

In [ ]:
cv_df.head()

# **BPlevel**

In [ ]:
# Defining a function to assign blood pressure levels
def bp_level(row):
    if row['sysBP'] < 120 or row['diaBP'] < 80:
        return 1  # Normal level
    elif (120 <= row['sysBP'] < 130) or row['diaBP'] < 80:
        return 2  # Elevated level
    elif (130 <= row['sysBP'] < 140) or (80 <= row['diaBP'] < 90):
        return 3  # High BP stage 1
    elif (140 <= row['sysBP'] < 180) or (90 <= row['diaBP'] < 120):
        return 4  # High BP stage 2
    else:
        return 5  # Hypertensive crisis

# Create the 'BPLevel' column using the function
cv_df['BPLevel'] = cv_df.apply(bp_level, axis=1)

# Remove the 'sysBP' and 'diaBP' columns
cv_df.drop(columns=['sysBP', 'diaBP'], inplace=True)

# Checking if the 'BPLevel' column is created properly
cv_df.head()

# **DiabetesLevel**

In [ ]:
# Define a function to assign diabetes levels
def diabetes_level(glucose):
    if glucose < 53:
        return 1  # Severe Hypoglycemia
    elif 53 <= glucose < 70:
        return 2  # Hypoglycemia
    elif 70 <= glucose < 125:
        return 3  # Normal
    elif 125 <= glucose < 200:
        return 4  # Pre Diabetic
    else:
        return 5  # Severe Diabetes

# Create the 'DiabetesLevel' column using the function
cv_df['DiabetesLevel'] = cv_df['glucose'].apply(diabetes_level)

# Remove the 'diabetes' and 'glucose' columns
cv_df.drop(columns=['diabetes', 'glucose'], inplace=True)

# Checking if the 'DiabetesLevel' column is created properly
cv_df.head()


**Checking correlation for feature removal:**

In [ ]:
#Plotting correlation matrix using sns heatmap
corr_matrix= cv_df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corr_matrix,annot=True,cmap='coolwarm')
plt.title("Correlation between the variables of the dataset")
plt.show()

**There is no high correlation between majority variables but there for majority of the variables but there is a high correlation between "prevalentHyp" and "BPLevel". Here i will remove "prevalentHyp" because this is somehow direct related with "BPLevel" in mediacal terms.**

In [ ]:
# Remove columns with high correlation
cv_df.drop('prevalentHyp', axis=1, inplace=True)

This will reduce variables that do not contribute much in predicting the target variables

# **Checking the distribution of the data:**

In [ ]:
# Creating a list of all the independent variables
independent_cols=list(set(cv_df.columns)-{'TenYearCHD'})


In [ ]:
n=1
plt.figure(figsize=(14,30))
for i in independent_cols:
  plt.subplot(12,4,n)
  n= n+1
  sns.distplot(cv_df[i],color='teal')
  plt.title(i)
  plt.tight_layout()

As we can see from the distribution, there is a high class imbalance for the columns BPMeds and prevalentStroke, so they won't be able to impact the prediction of the target variable much and therefore we'll delete them.

From the EDA process we also saw that Education is not a great contributing factor, therefore I'll remove the education column also.

In [ ]:
#Removing useless columns
cv_df.drop(columns={'BPMeds','prevalentStroke','education'},axis=1,inplace=True)

# **Dealing with class imbalance:**

In [ ]:
#Checking for class imbalance for the target variable
cv_df['TenYearCHD'].value_counts()

In [ ]:
# Plotting the pie chart to check the balance in the dataset.

plt.figure(figsize=(7,5), dpi=100)
proportion = cv_df['TenYearCHD'].value_counts()
labels = ['SAFE','AT RISK']
plt.title('Proportion of Safe and at Risk for Target Feature')
plt.pie(proportion, explode=(0,0.2),labels=labels, shadow = True, autopct = '%1.1f%%')
plt.legend()
plt.show()

As we can see there is high class Imbalance

# **Handling Imbalance data **

Handling imbalance of target variable using SMOTE(Synthetic Minority Oversampling Technique)

In [ ]:
# Creating the dataset for the independent and dependent variables.
X = cv_df.drop('TenYearCHD', axis=1)
Y = cv_df['TenYearCHD'].reset_index(drop=True)

# Applying the SMOTE technique to solve class imbalance
smote = SMOTE(sampling_strategy='minority')
X_resampled, Y_resampled = smote.fit_resample(X, Y)

# Displaying the first few rows of the resampled independent variables (X_resampled)
X_resampled.head()


In [ ]:
Y_resampled.value_counts()

**Class imbalance is now removed.**

# **Splitting the Data :**

In [ ]:
#Splitting the data
X_train,X_test,Y_train,Y_test = train_test_split(X_resampled,Y_resampled,test_size=0.25,random_state=12)

# **Feature Scaling :**

Feature Scaling is a technique to standardize the independent features present in the data in a fixed range. It is performed during the data pre-processing to handle highly varying magnitudes or values or units. If feature scaling is not done, then a machine learning algorithm tends to weigh greater values, higher and consider smaller values as the lower values, regardless of the unit of the values.

**StandardScaler is used to resize the distribution of values ​​so that the mean of the observed values ​​is 0 and the standard deviation is 1.**

In [ ]:


def scale_numeric_columns(data):
    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Scale only the numeric columns
    numeric_data = data.select_dtypes(include=['number'])

    # Fit and transform the scaler on the numeric data
    scaled_data = scaler.fit_transform(numeric_data)

    # Create a DataFrame with scaled values and original column names
    scaled_df = pd.DataFrame(scaled_data, columns=numeric_data.columns)

    return scaled_df

In [ ]:
#Scaling the independent dataset
scaled_X_train = scale_numeric_columns(X_train)
scaled_X_test = scale_numeric_columns(X_test)

# **Performance Metrics**

Different performance metrics are used to evaluate machine learning model. Based on our task we can choose our performance metrics. Since our task is of classification and that too binary class classification, whether client will or will not subscribe for deposits.

Here we will be using AUC ROC

ROC also known as Receiver Operating Characteristics, shows the performance of binary class classifiers across the range of all possible thresholds plotting between true positive rate and 1-false positive rate.

AUC measures the likelihood of two given random points, one from positive and one from negative, the classifier will rank the positive points above negative points. AUC-ROC is popular classification metric that presents the advantage of being independent of false positive or negative points.

Secondary Performance Metrics

Macro-F1 Score: F1 score is the harmonic mean between Precision and Recall. Macro F1 score is used to know how our model works in overall dataset.

Confusion Matrix: This matrix gives the count of true negative, true positive, false positive and false negative data points.

In [ ]:


def model_evaluator(actual, preds, ml_model, mode):
    """Evaluate a machine learning model and display metrics."""
    # Confusion matrix
    cm = confusion_matrix(actual, preds)
    print("Confusion Matrix:\n", cm, '\n')
    sns.heatmap(cm, annot=True, cmap='coolwarm', fmt='d')
    plt.xlabel('Predicted Labels')
    plt.ylabel('Actual Labels')
    plt.title(f'Confusion Matrix for {ml_model} on the {mode} set')
    plt.show()

    # ROC AUC score
    roc_auc = roc_auc_score(actual, preds)
    print('\nROC AUC Score:', roc_auc)

    # Classification report
    print('\nClassification Report:\n')
    target_names = ['Class 0', 'Class 1']
    print(classification_report(actual, preds, target_names=target_names))

def model_pipeline(X_train, X_test, Y_train, Y_test, ml_model, param_grid=None, kind='evaluate'):
    """Train and evaluate machine learning models."""
    global model

    # Logistic Regression
    if ml_model == 'Logistic Regression':
        model = LogisticRegression(random_state=12)

    # Decision Tree, Random Forest, Gradient Boosting
    elif ml_model in ['Decision Tree Classifier', 'Random Forest Classifier', 'Gradient Boosting Classifier']:
        model_init = {
            'Decision Tree Classifier': DecisionTreeClassifier(),
            'Random Forest Classifier': RandomForestClassifier(),
            'Gradient Boosting Classifier': GradientBoostingClassifier()
        }[ml_model]

        gs_model = GridSearchCV(estimator=model_init, param_grid=param_grid, cv=5, scoring='roc_auc', verbose=True)
        gs_model.fit(X_train, Y_train)

        print("Best parameters for", ml_model, ":", gs_model.best_params_, '\n')

        model = gs_model.best_estimator_
    else:
        print("Enter correct model name: Logistic Regression, Decision Tree Classifier, Random Forest Classifier, or Gradient Boosting Classifier.")

    model.fit(X_train, Y_train)

    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    if kind == 'evaluate':
        print("1. Train set evaluation:")
        model_evaluator(Y_train, train_predictions, ml_model, 'Train')
        print("\n2. Test set evaluation:")
        model_evaluator(Y_test, test_predictions, ml_model, 'Test')

    elif kind == 'model_explainability':
        return model

# Example usage:
# Assuming X_train, X_test, Y_train, and Y_test are defined earlier


param_grid_dt = {
    'max_depth': [4, 6, 8, 10],
    'min_samples_split': [5, 10, 20, 30, 40, 50],
    'min_samples_leaf': [5, 10, 15, 20]
}

param_grid_rf = {
    'n_estimators': [50, 65, 80, 95, 120],
    'max_depth': [3, 5, 7, 9, 10]
}

param_grid_gb = {
    'n_estimators': [80, 100],
    'max_depth': [5, 7, 8],
    'learning_rate': [0.001, 0.01, 0.05]
}




In [ ]:
model_pipeline(X_train, X_test, Y_train, Y_test, ml_model='Logistic regression')

In [ ]:
model_pipeline(X_train, X_test, Y_train, Y_test, ml_model='Decision Tree Classifier', param_grid=param_grid_dt, kind='evaluate')

In [ ]:
model_pipeline(X_train, X_test, Y_train, Y_test, ml_model='Random Forest', param_grid=param_grid_rf, kind='evaluate')

In [ ]:
model_pipeline(X_train, X_test, Y_train, Y_test, ml_model='Gradient Boosting Classifier', param_grid=param_grid_gb, kind='evaluate')

# **Model explainability:**

In [ ]:
#Installing the shap library
!pip install shap

In [ ]:
#Importing the SHAP library
import shap

In [ ]:
#Creating an object for the logistic regression model
lr_classifier = model_pipeline(X_train,X_test,Y_train,Y_test, ml_model='Logistic Regression',kind='model_explainability')

In [ ]:
#Plotting the shap summary plot
explainer_shap = shap.Explainer(model=model, masker=X_train)
shap_values = explainer_shap.shap_values(X_train)
shap.summary_plot(shap_values,X_train,feature_names=X.columns)

In [ ]:
def feature_importance(model):
  features = X.columns
  importances = model.feature_importances_
  indices = np.argsort(importances)
  plt.figure(figsize=(10, 10))
  plt.title('Feature Importance')
  plt.barh(range(len(indices)), importances[indices], color='blue', align='center')
  plt.yticks(range(len(indices)), [features[i] for i in indices])
  plt.xlabel('Relative Importance')
  plt.show()


In [ ]:
Dt_clf=model_pipeline(X_train, X_test, Y_train, Y_test, ml_model='Decision Tree Classifier',
                          param_grid={'max_depth': [4, 6, 8, 10],
                                      'min_samples_split': [5, 10, 20, 30, 40, 50],
                                      'min_samples_leaf': [5, 10, 15, 20]},
                          kind='model_explainability')

In [ ]:
#Plotting the feature importance for Decision tree classifier
feature_importance(Dt_clf)

In [ ]:
rf_clf = model_pipeline(X_train, X_test, Y_train, Y_test, ml_model='RandomForestClassifier',
                           param_grid={'n_estimators': [50, 65, 80, 95, 120],
                                       'max_depth': [3, 5, 7, 9, 10]},
                           kind='model_explainability')

In [ ]:
feature_importance(rf_clf)

In [ ]:
gb_clf = model_pipeline(X_train, X_test, Y_train, Y_test, ml_model='GradientBoostingClassifier',
                           param_grid={'n_estimators': [80, 100],
                                       'max_depth': [5, 7, 8],
                                       'learning_rate':[0.001, 0.01, 0.05]},
                           kind='model_explainability')

In [ ]:
feature_importance(gb_clf)

## **Summary :**

1.   Males are having high chances of CHD.
2.   Smoking increases CHD chances.
3.   Age between 47 to 65 years increase risk factor for CHD.
4.   People taking BPMeds also has high chances of CHD.
5.   Prevalent Stroke increases risk factors for CHD.
6.   Prevalent Hypertension also increases CHD chances in future.
7.   Diabetes also impact on positive CHD in future.





**Results from ML model :**

1.   Logistic regression gives a ROCAUC score of 0.69022 on the testing set.
2.   Decision tree model gives a ROCAUC score of 0.68495 on the testing set.
     This is the worst performing model
3.   Random Forest Classifier model gives a ROCAUC score of 0.77373 on the
     testing set.
4.   Gradient Boosting Classifier model gives a ROCAUC score of 0.8255 on the
     testing set.This is the best performing model
5.   Model explainability has been achieved by SHAP library's summary plot and
     an attribute called feature_importances_ of the tree based algorithms.
6.   Total cholestrol and age are the two most important factors to predict the
     CHD risk factor.

